In [1]:
!pip install trl
!pip install levenshtein
!pip install bitsandbytes
!pip install kaggle
!pip install kagglehub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
from datasets import load_dataset, Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from trl import GRPOConfig, GRPOTrainer
import datetime
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig,
    PrinterCallback,
)
from tqdm import tqdm
import torch
import time
import transformers
import pandas as pd
import numpy as np
from Levenshtein import ratio as levenshtein_ratio
transformers.set_seed(42)

In [4]:
class CFG:
    MAX_TRAIN = 100
    MAX_TOKENS = 2048
    NUM_GENERATIONS = 4
    USE_PEFT = True
    BATCH_SIZE = 1
    MAX_STEPS = 80
    BETA = 0.04
    LR = 1.e-5
    model_name = '/root/.cache/kagglehub/models/deepseek-ai/deepseek-r1/transformers/deepseek-r1-distill-qwen-1.5b/1' # モデル名
    splitter = '< | Assistant | >'
    step_count = 10
    DEBUG = False

In [5]:
import kagglehub
# Download latest version
path = kagglehub.model_download("deepseek-ai/deepseek-r1/transformers/deepseek-r1-distill-qwen-1.5b")
print("Path to model files:", path)


  0%|          | 0.00/3.31G [00:00<?, ?B/s]



100%|██████████| 181/181 [00:00<00:00, 96.8kB/s]




100%|██████████| 14.2k/14.2k [00:00<00:00, 7.81MB/s]



  0%|          | 0.00/6.71M [00:00<?, ?B/s]


100%|██████████| 1.48k/1.48k [00:00<00:00, 991kB/s]

  0%|          | 1.00M/3.31G [00:00<21:53, 2.70MB/s]




  0%|          | 0.00/759k [00:00<?, ?B/s]





  0%|          | 0.00/679 [00:00<?, ?B/s]

100%|██████████| 679/679 [00:00<00:00, 61.4kB/s]




100%|██████████| 1.04k/1.04k [00:00<00:00, 763kB/s]

  0%|          | 3.00M/3.31G [00:00<07:51, 7.53MB/s]
  0%|          | 9.00M/3.31G [00:00<02:42, 21.8MB/s]
  0%|          | 15.0M/3.31G [00:00<01:52, 31.4MB/s]

 15%|█▍        | 1.00M/6.71M [00:00<00:02, 2.82MB/s]


100%|██████████| 759k/759k [00:00<00:00, 2.16MB/s]

  1%|          | 22.0M/3.31G [00:00<01:27, 40.2MB/s]

100%|██████████| 6.71M/6.71M [00:00<00:00, 12.6MB/s]

  1%|          | 27.0M/3.31G [00:00<01:29, 39.4MB/s]
  1%|          | 32.0M/3.31G [00:01<01:25, 41.1MB/s]



100%|██████████| 2.99k/2.99k [00:00<00:00, 2.09MB/s]

  1%|          | 38.0M/3.31G [00:01<01:15, 46.7MB/s]
  1%|▏         | 43.0M/3.31G [00:01<01:15, 46.3MB/s]
  1%|▏         | 49.0M/3.31G [00:01<01:11, 49.2MB/s]
  2%|▏         | 55.0M/3.31G [00:01<01:07, 52.2MB/s]
  2%|▏         | 62.0M/3.31G [00:01<01:05, 53.5MB/s]
  2%|▏         | 69.0M/3.31G [00:01<01:01, 56.8MB/s]
  2%|▏         | 75.0M/3.31G [00:01<01:13, 47.3MB/s]
  2%|▏         | 82.0M/3.31G [00:02<01:19, 43.8MB/s]
  3%|▎         | 89.0M/3.31G [00:02<01:12, 47.7MB/s]
  3%|▎         | 96.0M/3.31G [00:02<01:07, 50.9MB/s]
  3%|▎         | 103M/3.31G [00:02<01:04, 53.1MB/s] 
  3%|▎         | 109M/3.31G [00:02<01:02, 54.7MB/s]
  3%|▎         | 116M/3.31G [00:02<01:01, 56.0MB/s]
  4%|▎         | 123M/3.31G [00:02<00:58, 58.4MB/s]
  4%|▍         | 129M/3.31G [00:03<01:12, 47.0MB/s]
  4%|▍         | 136M/3.31G [00:03<01:07, 50.3MB/s]
  4%|▍         | 142M/3.31G [00:03<01:09, 48.7MB/s]
  4%|▍         | 148M/3.31G [00:03<01:12, 47.1MB/

Path to model files: /root/.cache/kagglehub/models/deepseek-ai/deepseek-r1/transformers/deepseek-r1-distill-qwen-1.5b/1


In [7]:
import re
def extract_boxed_text(text):
    pattern = r'\\boxed{(.*?)}'
    matches = re.findall(pattern, text)
    if not matches:
        return ""
    for match in matches[::-1]:
        if match != "":
            return match
    return ""

In [8]:
def is_valid_answer(s):
    try:
        if float(s) == int(s):
            i = int(s)
            return 0<=i<1000
        else:
            return False
    except ValueError:
        return False

In [9]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("artemgoncarov/math-problems-imo")
print("Path to dataset files:", path)

100%|██████████| 63.0M/63.0M [00:01<00:00, 59.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/artemgoncarov/math-problems-imo/versions/1


In [10]:
df = pd.read_parquet('/root/.cache/kagglehub/datasets/artemgoncarov/math-problems-imo/versions/1')
df = df.reset_index().rename({'index': 'id'}, axis=1)
df['answer'] = df['solution'].map(extract_boxed_text)
mask = df['answer'].map(is_valid_answer)
df = df[mask]
df = df.iloc[:CFG.MAX_TRAIN]
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'problem', 'solution', 'answer', '__index_level_0__'],
        num_rows: 90
    })
    test: Dataset({
        features: ['id', 'problem', 'solution', 'answer', '__index_level_0__'],
        num_rows: 10
    })
})

In [11]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name, trust_remote_code=True, padding_side="left")

In [12]:
def create_prompt(sample):
    question = sample['problem']
    chat = [{"role": "system", "content": "A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>"},
            {"role": "user", "content": question + ' Return final answer within \\boxed{}, after taking modulo 1000.'},]
    sample['prompt'] = tokenizer.apply_chat_template(
        conversation=chat,
        tokenize=False,
        add_generation_prompt=True
    )
    return sample

In [13]:
def format_reward_func(completions, **kwargs):
    pattern = r"^<think>.*?</think>.*?\\boxed{(.*?)}.*?$"
    matches = [re.match(pattern, content, re.DOTALL) for content in completions]
    return [1.0 if match else 0.0 for match in matches]

def accuracy_reward_func(completions, answer, **kwargs):
    contents = [extract_boxed_text(completion) for completion in completions]
    return [1.0 if c == str(gt) else 0.0 for c, gt in zip(contents, answer)]

def levenshtein_reward_func(completions, solution, **kwargs):
    res = []
    for completion, sol in zip(completions, solution):
        if '</think>' in completion:
            t = completion.split('</think>')[-1]
            res.append(levenshtein_ratio(t, sol))
        else:
            res.append(0.0)
    return res

reward_functions = {'formatting': format_reward_func, 'accuracy': accuracy_reward_func, 'solution_quality': levenshtein_reward_func}

In [14]:
device_map = 'auto'
if CFG.USE_PEFT:
    compute_dtype = getattr(torch, "float16")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )
    original_model = AutoModelForCausalLM.from_pretrained(CFG.model_name,
                                                device_map=device_map,
                                                quantization_config=bnb_config,
                                                trust_remote_code=True)
else:
    original_model = AutoModelForCausalLM.from_pretrained(CFG.model_name,
                                                device_map=device_map,
                                                trust_remote_code=True)


In [15]:
dataset = dataset.map(create_prompt)

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [16]:
def gen(model, text, max_tokens):
    model_input = tokenizer(text, return_tensors='pt').to(model.device)
    model.eval()
    with torch.no_grad():
        tok = model.generate(**model_input, max_new_tokens=max_tokens, pad_token_id=tokenizer.pad_token_type_id)
        outputs = []
        for i in range(len(tok)):
            res = tokenizer.decode(tok[i], skip_special_tokens=True)
            output = res.split(CFG.splitter)[-1]
            outputs.append(output)
        return outputs[0] if len(outputs) == 1 else outputs

def evaluate_rewards(model, dataset, reward_functions: dict[str, callable], max_tokens: int, num_generations: int):
    completions = []
    other_info = []
    for example in tqdm(dataset):
        txt = example['prompt']
        kw = {k: v for k, v in example.items() if k not in {'prompt', 'completion'}}
        for _ in range(num_generations):
            other_info.append(kw)
        completion = gen(model, [txt] * num_generations, max_tokens)
        if isinstance(completion, str):
            completions.append(completion)
        else:
            completions += completion

    kwargs = {k: [d[k] for d in other_info] for k in other_info[0].keys()}
    res = {}
    for nm, reward_func in reward_functions.items():
        v = reward_func(completions=completions, **kwargs)
        print(nm, np.mean(v))
        res[nm] = np.mean(v)
    return res

if not CFG.DEBUG:
    original_rewards = evaluate_rewards(model=original_model, dataset=dataset['test'], reward_functions=reward_functions, max_tokens=CFG.MAX_TOKENS, num_generations=CFG.NUM_GENERATIONS)


100%|██████████| 10/10 [15:38<00:00, 93.89s/it]

formatting 0.0
accuracy 0.5
solution_quality 0.2811005776380352


In [17]:
dtstr = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
output_directory = f"./DEEPSEEK-GRPO-{dtstr}"
training_args = GRPOConfig(
    output_dir=output_directory,
    learning_rate=CFG.LR,
    per_device_train_batch_size=CFG.BATCH_SIZE,
    gradient_accumulation_steps=1,
    max_steps=CFG.MAX_STEPS,
    max_completion_length=CFG.MAX_TOKENS, #8192
    num_generations=CFG.NUM_GENERATIONS,
    beta=CFG.BETA,
    logging_steps=CFG.step_count,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=CFG.step_count,
    report_to="none",
    overwrite_output_dir='True',
)

In [18]:
if CFG.USE_PEFT:
    peft_config = LoraConfig(
        r=32,
        lora_alpha=32,
        target_modules=[
            'q_proj',
            'k_proj',
            'v_proj',
            'dense'
        ],
        bias="none",
        lora_dropout=0.05,
        task_type="CAUSAL_LM",
    )

In [19]:
if CFG.USE_PEFT:
    trainer = GRPOTrainer(
        model=original_model,
        reward_funcs=list(reward_functions.values()),
        args=training_args,
        train_dataset=dataset['train'],
        peft_config=peft_config,
        callbacks=[PrinterCallback()]
    )
else:
    trainer = GRPOTrainer(
        model=original_model,
        reward_funcs=list(reward_functions.values()),
        args=training_args,
        train_dataset=dataset['train'],
        callbacks=[PrinterCallback()]
    )

trainer.train()

Step,Training Loss
10,-0.000000
20,-0.000000
30,0.000000
40,-0.000000
50,-0.000000
60,-0.000000
70,-0.000000
80,-0.000000


{'loss': -0.0, 'grad_norm': 0.04268573969602585, 'learning_rate': 8.750000000000001e-06, 'completion_length': 1666.1, 'rewards/format_reward_func': 0.375, 'rewards/accuracy_reward_func': 0.25, 'rewards/levenshtein_reward_func': 0.16982566937804222, 'reward': 0.7948256760835648, 'reward_std': 0.40151107385754586, 'kl': -5.543231964111328e-06, 'epoch': 0.1111111111111111}
{'loss': -0.0, 'grad_norm': 0.0, 'learning_rate': 7.500000000000001e-06, 'completion_length': 1315.6, 'rewards/format_reward_func': 0.4, 'rewards/accuracy_reward_func': 0.375, 'rewards/levenshtein_reward_func': 0.1898331694304943, 'reward': 0.9648331589996815, 'reward_std': 0.4784363716840744, 'kl': -6.99758529663086e-06, 'epoch': 0.2222222222222222}
{'loss': 0.0, 'grad_norm': 0.04810426011681557, 'learning_rate': 6.25e-06, 'completion_length': 1323.575, 'rewards/format_reward_func': 0.55, 'rewards/accuracy_reward_func': 0.475, 'rewards/levenshtein_reward_func': 0.2830824114382267, 'reward': 1.3080824375152589, 'reward_

TrainOutput(global_step=80, training_loss=-1.6083267837530001e-07, metrics={'train_runtime': 11500.4713, 'train_samples_per_second': 0.007, 'train_steps_per_second': 0.007, 'total_flos': 0.0, 'train_loss': -1.6083267837530001e-07})

In [ ]:
if CFG.USE_PEFT:
    print('Loading trained model')
    CHKPT = CFG.MAX_STEPS
    adapter_model_name = f'{output_directory}/checkpoint-{CHKPT}/'
    new_model = PeftModel.from_pretrained(original_model, adapter_model_name)
else:
    new_model = original_model

rewards = evaluate_rewards(model=new_model, dataset=dataset['test'], reward_functions=reward_functions, max_tokens=CFG.MAX_TOKENS, num_generations=CFG.NUM_GENERATIONS)
rewards

Loading trained model


 80%|████████  | 8/10 [16:16<03:05, 92.97s/it] 